In [3]:
import nltk
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la

### Load data

In [12]:
data_folder = './data/'
output_folder = './output/'

data_path = data_folder + 'tw_hurricane_data.json'
map_path = data_folder + 'tweet_document_ids_map.csv'


with open(data_path) as df:
    lines = df.readlines()

In [11]:
print("Total number of tweets: {}".format(len(lines)))

Total number of tweets: 4000


In [13]:
lines[0]

'{"created_at": "Fri Sep 30 18:39:08 +0000 2022", "id": 1575918182698979328, "id_str": "1575918182698979328", "full_text": "So this will keep spinning over us until 7 pm\\u2026go away already. #HurricaneIan https://t.co/VROTxNS9rz", "truncated": false, "display_text_range": [0, 76], "entities": {"hashtags": [{"text": "HurricaneIan", "indices": [63, 76]}], "symbols": [], "user_mentions": [], "urls": [], "media": [{"id": 1575918178261254162, "id_str": "1575918178261254162", "indices": [77, 100], "media_url": "http://pbs.twimg.com/media/Fd7JO8pXwBI9HPw.jpg", "media_url_https": "https://pbs.twimg.com/media/Fd7JO8pXwBI9HPw.jpg", "url": "https://t.co/VROTxNS9rz", "display_url": "pic.twitter.com/VROTxNS9rz", "expanded_url": "https://twitter.com/suzjdean/status/1575918182698979328/photo/1", "type": "photo", "sizes": {"small": {"w": 521, "h": 680, "resize": "fit"}, "thumb": {"w": 150, "h": 150, "resize": "crop"}, "medium": {"w": 919, "h": 1200, "resize": "fit"}, "large": {"w": 1284, "h": 1677, 

In [49]:
import json

t = json.loads(lines[0])

{'created_at': 'Fri Sep 30 18:39:08 +0000 2022', 'id': 1575918182698979328, 'id_str': '1575918182698979328', 'full_text': 'So this will keep spinning over us until 7 pm…go away already. #HurricaneIan https://t.co/VROTxNS9rz', 'truncated': False, 'display_text_range': [0, 76], 'entities': {'hashtags': [{'text': 'HurricaneIan', 'indices': [63, 76]}], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1575918178261254162, 'id_str': '1575918178261254162', 'indices': [77, 100], 'media_url': 'http://pbs.twimg.com/media/Fd7JO8pXwBI9HPw.jpg', 'media_url_https': 'https://pbs.twimg.com/media/Fd7JO8pXwBI9HPw.jpg', 'url': 'https://t.co/VROTxNS9rz', 'display_url': 'pic.twitter.com/VROTxNS9rz', 'expanded_url': 'https://twitter.com/suzjdean/status/1575918182698979328/photo/1', 'type': 'photo', 'sizes': {'small': {'w': 521, 'h': 680, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 919, 'h': 1200, 'resize': 'fit'}, 'large': {'w': 1284, 'h': 1677, 'resize

In [51]:
t

{'created_at': 'Fri Sep 30 18:39:08 +0000 2022',
 'id': 1575918182698979328,
 'id_str': '1575918182698979328',
 'full_text': 'So this will keep spinning over us until 7 pm…go away already. #HurricaneIan https://t.co/VROTxNS9rz',
 'truncated': False,
 'display_text_range': [0, 76],
 'entities': {'hashtags': [{'text': 'HurricaneIan', 'indices': [63, 76]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1575918178261254162,
    'id_str': '1575918178261254162',
    'indices': [77, 100],
    'media_url': 'http://pbs.twimg.com/media/Fd7JO8pXwBI9HPw.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/Fd7JO8pXwBI9HPw.jpg',
    'url': 'https://t.co/VROTxNS9rz',
    'display_url': 'pic.twitter.com/VROTxNS9rz',
    'expanded_url': 'https://twitter.com/suzjdean/status/1575918182698979328/photo/1',
    'type': 'photo',
    'sizes': {'small': {'w': 521, 'h': 680, 'resize': 'fit'},
     'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 919, 'h': 120

In [67]:
# Output format: Tweet | Username | Date | Hashtags | Likes | Retweets | Url
print('Output format: Tweet | Username | Date | Hashtags | Likes | Retweets | Url\n')

for i in range(4):
    tweet = json.loads(lines[i])

    print('Tweet : full_text : \t\t\t' + repr(tweet['full_text']))
    print('Username : user.name : \t\t\t{}'.format(tweet['user']['name']))
    print('Date : created_at : \t\t\t{}'.format(tweet['created_at']))
    print('Hashtags : entities.hashtags.text : \t{}'.format(tweet['entities']['hashtags'][0]['text']))
    print('Likes : favorite_count : \t\t{}'.format(tweet['favorite_count']))
    print('Retweets : retweet_count : \t\t{}'.format(tweet['retweet_count']))
    try:
        print('Url : entities.media.url : \t\t{}\n'.format(tweet['entities']['media'][0]['url']))
    except:
        print('Url : entities.media.url : \t\tnull\n')

Output format: Tweet | Username | Date | Hashtags | Likes | Retweets | Url

Tweet : full_text : 			'So this will keep spinning over us until 7 pm…go away already. #HurricaneIan https://t.co/VROTxNS9rz'
Username : user.name : 			Suz👻
Date : created_at : 			Fri Sep 30 18:39:08 +0000 2022
Hashtags : entities.hashtags.text : 	HurricaneIan
Likes : favorite_count : 		0
Retweets : retweet_count : 		0
Url : entities.media.url : 		https://t.co/VROTxNS9rz

Tweet : full_text : 			'Our hearts go out to all those affected by #HurricaneIan. We wish everyone on the roads currently braving the conditions safe travels. 💙'
Username : user.name : 			Lytx
Date : created_at : 			Fri Sep 30 18:39:01 +0000 2022
Hashtags : entities.hashtags.text : 	HurricaneIan
Likes : favorite_count : 		0
Retweets : retweet_count : 		0
Url : entities.media.url : 		null

Tweet : full_text : 			'Kissimmee neighborhood off of Michigan Ave. \n#HurricaneIan https://t.co/jf7zseg0Fe'
Username : user.name : 			Christopher Heath
Date